# Adressefunksjoner
> Funksjoner knyttet til adresseavledning fra FREG systemet

## Introduksjon
I forbindelse med overgangen til FREG har skatteetaten koblet seg på matrikkelens API. 
Dette har gjort at FREG nå inneholder betraktelig mer informasjon om personers adresser enn tidligere. 
Det har også gjort det enklere enn noen gang å koble sammen personer og bygninger. 

Denne notatboken inneholder funksjoner som knytter seg til det å avlede adresseinformasjon.
Dette kan gjøres med forskjellige formål. 

## adr_26
Adresse 26 er en kombinasjon av 
```kommunenummer - gate/gårdsnummer - husnummer/bruksnummer -  bokstav/festenummer - undernummer - bruksenhetsnummer```
Sammen identifiserer disse alle godkjente(?) bruksenheter. 
I FREG, som i de fleste andre registre, eksisterer ikke adr_26 som en egen variabel og må derfor sammenstilles. 
Dette kan gjøres med en av disse funksjonene.

## To offisielle adresser
I Norge opererer vi med to offisielle adresser. 
Den ene er matrikkeladressen, som idenitfiserer en eiendom; den andre er vegadressen, som identifiserer bygninger. 
Et adressepunkt har ikke to adresser samtidig, men det er opp til de enkelte kommunenene om de bruker veg- eller matrikkeladresser.
I dag har alle kommuner gått over til å bruke vegadresser – men dette er arbeid som tar tid. 
Derfor er det slik at et bygg enten har en matrikkel- eller en vegadresse, og det er blanding av begge innad i kommunenene. 

FREG representerer de to adressene i hvert sitt element i alle adresseelementer som er knyttet til en person.
Heldigvis er strukturen på disse feltene like.

In [ ]:
# default_exp freg_adresse_funcs
#hide
import os
import sys
import string

from ssb_sparktools.utils import utils as stutils 
stutils.set_runpath('stat-freg')
sys.path.append(os.path.abspath(os.getcwd()))
from ssb_sparktools.processing import processing as stproc

import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import SQLContext
from pyspark.sql import Window
from pyspark.sql.functions import broadcast
from pyspark.sql.types import StringType

import toml
paths = toml.load('src/config/filstier.toml')

## generate_adresse_info
Som beskrevet over, er FREG-adresseinformasjon pakket ned i forskjellige informasjonselementer og deretter forskjellige underelementer. 
Denne funksjonen tar som input et informasjonselement med adresseinformasjon, gitt at alle adresseelementer inneholder underelementer, som inneholder adresseinformasjonen.
Får å få tak i sistnevnte må vi foreta en sekundær utpakking, begrenset til innenfor denne funksjonen.

Matrikkel- og vegadresser prosesseres separat, før de kombineres på slutten av funksjonen. 

Ett av kravene til innputten, er `rootvar`, som er en liste av variabler som skal følge adresseinformasjonen gjennom prossessen.
For å kunne koble adresseinformasjonen tilbake til det opprinnelige datasettet trenger du i alle fall én variabel i denne listen, og 
som regel vil dette være `folkeregisteridentifikator`.
Dersom en person kan være representert med mer enn en adresse i datasettet, må det også inkluderes en variabel som i kombinasjon med `folkeregisteridentifikator`, unikt identifiserer en adresseinstans.
I vanlig produksjon vil dette mest sannsylig være en `*_id`-variabel. 

Som beskrevet innledningsvis, har veg- og matrikkeladresser forskjellige navn på de forskjellige posisjonenene i den numeriske adressen. 
For å sammenstille til en enhetlig adresseinformasjon må disse forskjellene harmoniseres. I funksjonen gjøres dette før datasettene er kombinert.
I tillegg til harmoniseringen av kolonnenavn, fylles ikke-utfylte variabler med `0`, for å sikre at totalantallet for den numeriske adressen er stabil.

Her er også et filter som fjerner tomme adresser, som er adresser der alle 26 posisjoner består av `0`.

Funksjonen retunerer et datasett med de variablene du oppgir i `rootvar`, som koblingsnøkler.
I tillegg til den kombinerte adr_26-variabelen, retuneres også den dekonstruerte adresseinformasjonen. 

In [ ]:
#export
def husnummer_omkoding(husbokstav):
    import string
    asc = string.ascii_uppercase + 'ÆØÅ'
    if husbokstav==None:
        nummer='0000'
    else:
        husbokstav = str.upper(husbokstav[:1])
        if (husbokstav in asc):
            nonpad = asc.index(husbokstav)+1            
            nummer = '99'+'{0:02d}'.format(nonpad)
        else:
            nummer = '9999'
    return nummer

In [ ]:
#exports
def get_adresse_info(df , rootvar=['folkeregisteridentifikator']):
    """
    Funksjon for å utlede full adresse informasjon. 
    """
    from pyspark.sql.functions import udf
    husnummer_omkoding_udf = udf(husnummer_omkoding)
    
    adr = stproc.unpack_parquet(df, 
                                 rootvar=rootvar, 
                                 rootdf=False, 
                                 levels=1 )
    
    base_list = ['kommunenummer',
                 'gatenr_gaardsnr',
                 'husnr_bruksnr', 
                 'bokstav_festenr',
                 'undernummer',
                 'bruksenhetsnummer',
                 'bruksenhetstype',
                 'adresse_type'
                ]
    
    select_list = rootvar+base_list
    
    vegadresser = (adr['vegadresse']
                   .filter(F.col('adressekode').isNotNull())
                   .withColumn('husnr_bruksnr',F.lpad(F.col('adressenummer.husnummer'),4,'0'))
                   .withColumn('bokstav_festenr', husnummer_omkoding_udf(F.col('adressenummer.husbokstav')))
                   .withColumn('undernummer', F.lit('0000'))
                   .withColumn('gatenr_gaardsnr',F.lpad('adressekode',5,'0'))
                   .withColumn('bruksenhetsnummer',  F.lpad('bruksenhetsnummer',5,'0'))
                   .withColumn('adresse_type', F.lit('O'))
                   .select(select_list)
                   .dropDuplicates()
                   .fillna('0000', subset=['undernummer', 'bokstav_festenr','kommunenummer', 'Husnr_bruksnr'])
                   .fillna('00000', subset=['bruksenhetsnummer','gatenr_gaardsnr'])
                   .select('*', F.concat('kommunenummer', 'gatenr_gaardsnr', 'husnr_bruksnr', 'bokstav_festenr', 'undernummer' , 'bruksenhetsnummer').alias('adr_26'))
                   
                  )
    
    matrikkeladresser = (adr['matrikkeladresse']
                         .filter(F.col('matrikkelnummer').isNotNull())
                         .withColumn('kommunenummer',F.lpad(F.col('matrikkelnummer.kommunenummer'),4,'0'))
                         .withColumn('gatenr_gaardsnr', F.lpad(F.col('matrikkelnummer.gaardsnummer'),5,'0'))
                         .withColumn('husnr_bruksnr',F.lpad(F.col('matrikkelnummer.bruksnummer'),4,'0'))
                         .withColumn('undernummer',F.lpad(F.col('undernummer'),4,'0'))
                         .withColumn('bruksenhetsnummer',F.lpad('bruksenhetsnummer',5,'0'))
                         .withColumn('bokstav_festenr', F.lpad(F.col('matrikkelnummer.festenummer'),4,'0'))
                         .withColumn('adresse_type', F.lit('M'))
                         .select(select_list)
                         .dropDuplicates()
                         .fillna('0000', subset=['kommunenummer', 'husnr_bruksnr','bokstav_festenr', 'undernummer'])
                         .fillna('00000', subset=['bruksenhetsnummer','gatenr_gaardsnr'])
                         .select('*', F.concat('kommunenummer', 'gatenr_gaardsnr', 'husnr_bruksnr', 'bokstav_festenr', 'undernummer', 'bruksenhetsnummer' ).alias('adr_26'))
                        )
    ukjent_bosted = (adr['ukjentBosted']
                     .filter(F.col('bostedskommune').isNotNull())
                     .withColumnRenamed('bostedskommune','kommunenummer')
                         .withColumn('gatenr_gaardsnr', F.lit('00000'))
                         .withColumn('husnr_bruksnr',F.lit('0000'))
                         .withColumn('undernummer',F.lit('0000'))
                         .withColumn('bruksenhetsnummer',F.lit('00000'))
                         .withColumn('bokstav_festenr', F.lit('0000'))
                         .withColumn('adresse_type', F.lit('ukjent'))
                     .withColumn('bruksenhetstype', F.lit(None))
                     .select(select_list)
                     .select('*', F.concat('kommunenummer', 'gatenr_gaardsnr', 'husnr_bruksnr', 'bokstav_festenr', 'undernummer', 'bruksenhetsnummer' ).alias('adr_26'))
                    )
    
    adresse_info = vegadresser.union(matrikkeladresser).union(ukjent_bosted)
    
    adresse_info = adresse_info.withColumn('bruksenhetsnummer', F.when(F.col('bruksenhetsnummer')=='00000', F.lit(None)).otherwise((F.col('bruksenhetsnummer'))))

    return adresse_info

### TEST: get_adresse_info
Koden under dette punktet består av enhetstester (unit-tester) som er skrevet for å kontrollere funksjonaliteten til get_adresse_info-funksjonen.

Det vil gjennomgås grupperinger av tester for å beskrive hvilket formål disse testene dekker.
Dette er ikke det endelige settet med tester, men det er en start.

In [ ]:
#hide
spark = SparkSession.builder.getOrCreate() 
fil = os.getcwd()+'/tests/test_data/fregtestdata'
fregtest = spark.read.parquet(fil)
informasjons_element = stproc.unpack_parquet(fregtest, rootvar=['folkeregisteridentifikator'], rootdf=False, levels=1)
adresse_info = get_adresse_info(informasjons_element['bostedsadresse'], rootvar=['folkeregisteridentifikator', 'bostedsadresse_id'])
valid_col = ['folkeregisteridentifikator',
             'bostedsadresse_id',
             'kommunenummer',
             'gatenr_gaardsnr',
             'husnr_bruksnr', 
             'bokstav_festenr',
             'undernummer',
             'bruksenhetsnummer',
             'bruksenhetstype',
             'adresse_type',
             'adr_26']

Disse testene skal sjekke at funksjonen ikke fjerner data og at data inn samsvarer med data ut. 
De sjekker også at funksjonen klarer å velge korrekt antall enheter med henholdsvis veg- og matrikkeladresser som er kommet inn med data-innputten 

In [ ]:
#hide
assert adresse_info.count() == informasjons_element['bostedsadresse'].select('folkeregisteridentifikator','bostedsadresse_id','matrikkeladresse', 'vegadresse', 'ukjentBosted').dropDuplicates().count()
assert adresse_info.filter(F.col('adresse_type')=='M').count() == informasjons_element['bostedsadresse'].filter(F.col('matrikkeladresse').isNotNull()).count()
assert adresse_info.filter(F.col('adresse_type')=='O').count() == informasjons_element['bostedsadresse'].filter(F.col('vegadresse').isNotNull()).count()

Adr_26 og dens bestanddeler har en spesifikk lengde som sikrer at det endelige resultatet blir korrekt. 
Flere av komponentene krever derfor en padding for å oppnå denne lengden.
Testene i denne bolken sjekker at variablene får korrekt lengde.

In [ ]:
#hide
assert adresse_info.where(F.length(F.col('adr_26'))<26).count() == 0
assert adresse_info.where(F.length(F.col('kommunenummer'))<4).count() == 0
assert adresse_info.where(F.length(F.col('gatenr_gaardsnr'))<5).count() == 0
assert adresse_info.where(F.length(F.col('husnr_bruksnr'))<4).count() == 0
assert adresse_info.where(F.length(F.col('bokstav_festenr'))<4).count() == 0
assert adresse_info.where(F.length(F.col('undernummer'))<4).count() == 0
assert adresse_info.where(F.length(F.col('bruksenhetsnummer'))<5).count() == 0
assert adresse_info.where(F.length(F.col('adresse_type'))<1).count() == 0

husnummer_omkoding funksjonen koder om husbokstav til 4 siffer kode 

In [ ]:
assert husnummer_omkoding(None)== '0000'
assert husnummer_omkoding('A')== '9901'
assert husnummer_omkoding('Å')== '9929'
assert husnummer_omkoding('~')== '9999'

get_adresse_info-funksjonen skal ta variablene den mottar som del av rootvar og legge disse til resultatet.

In [ ]:
#hide
assert adresse_info.columns == valid_col

Variabelen `adresse_type` er en konstruert variabel. 
Vi har derfor en funksjon som skjekker at variabelen kun inneholder gyldige verdier.
`bruksenhetsnummer` og `bruksenhetstype` er i tillegg klassifiseringsvariabler. 
Av den grunn har vi tester som skjekker at funksjonen ikke introduserer ugyldige variabler.

In [ ]:
#hide
assert adresse_info.where(F.col('adresse_type').isin(['M', 'O','ukjent'])==False).count() == 0
assert adresse_info.where(F.col('bruksenhetsnummer').substr(0,1).isin(['0','H','U', 'L', 'K'])==False).count() == 0